In [1]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\users\crie072\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


pip install -U xlwt

pip install -U openpyxl

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

from pandas.tseries.offsets import MonthEnd

In [3]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200905


In [4]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [6]:
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202008_退会顧客\Cognos時系列")

#input
input_file = "個人会員残高_from_cognos.xlsx"
input_file2 = "age_code"
input_file3 = "cancel_code"

In [ ]:
#シート名取得
bk = pd.ExcelFile(data_dir/f"{input_file}")
print(bk.sheet_names)

退会顧客データデータの読込＆集計

In [173]:
df0 = pd.read_excel(data_dir/f"{input_file}", sheet_name = 0)
df1 = pd.read_excel(data_dir/f"{input_file}", sheet_name = 1)
df2 = pd.read_excel(data_dir/f"{input_file}", sheet_name = 2)
df3 = pd.read_excel(data_dir/f"{input_file}", sheet_name = 3)
df4 = pd.read_excel(data_dir/f"{input_file}", sheet_name = 4)
df5 = pd.read_excel(data_dir/f"{input_file}", sheet_name = 5)

In [174]:
df = pd.concat([df0,df1,df2,df3,df4,df5])
df.head()

,会員カード会社コード,カード提携方式コード,カード提携会社コード１＋２,カード種類コード,カードブランドコード,管轄部店コード,加入年度,加入年度区分（Ａ，Ｂ，Ｃ）,職業コード,年齢階層,性別コード,データ基準年月,活口座数,解約口座数
0,155,10,D0001,2,1,0,2006,A,3,7,2,201903,1,0
1,155,10,D0001,2,1,0,2006,A,3,7,2,201904,1,0
2,155,10,D0001,2,1,0,2006,A,3,7,2,201905,1,0
3,155,10,D0001,2,1,0,2006,A,3,7,2,201906,1,0
4,155,10,D0001,2,1,0,2006,A,3,7,2,201907,1,0


In [175]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(340749, 14)
Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            15754, 15755, 15756, 15757, 15758, 15759, 15760, 15761, 15762,
            15763],
           dtype='int64', length=340749)
Index(['会員カード会社コード', 'カード提携方式コード', 'カード提携会社コード１＋２', 'カード種類コード', 'カードブランドコード',
       '管轄部店コード', '加入年度', '加入年度区分（Ａ，Ｂ，Ｃ）', '職業コード', '年齢階層', '性別コード', 'データ基準年月',
       '活口座数', '解約口座数'],
      dtype='object')
会員カード会社コード        int64
カード提携方式コード        int64
カード提携会社コード１＋２    object
カード種類コード          int64
カードブランドコード        int64
管轄部店コード           int64
加入年度              int64
加入年度区分（Ａ，Ｂ，Ｃ）    object
職業コード            object
年齢階層              int64
性別コード             int64
データ基準年月           int64
活口座数              int64
解約口座数             int64
dtype: object


In [176]:
df = df.rename(columns={'会員カード会社コード':'kaisha_code',
                        'カード提携方式コード':'houshiki_code',
                        'カード提携会社コード１＋２':'teikeikaisha_code1_2',
                        'カード種類コード':'card_code',
                        'カードブランドコード':'brand_code',
                        '管轄部店コード':'buten_code',
                        '加入年度':'nendo',
                        '加入年度区分（Ａ，Ｂ，Ｃ）':'nendo_kubun',
                        '職業コード':'job_code',
                        '年齢階層':'age_code',
                        '性別コード':'gender_code',
                        'データ基準年月':'cp',
                        '活口座数':'active_account',
                        '解約口座数':'cancel_account'})
print(df.dtypes)

kaisha_code              int64
houshiki_code            int64
teikeikaisha_code1_2    object
card_code                int64
brand_code               int64
buten_code               int64
nendo                    int64
nendo_kubun             object
job_code                object
age_code                 int64
gender_code              int64
cp                       int64
active_account           int64
cancel_account           int64
dtype: object


In [ ]:
#味見

In [12]:
print(df['kaisha_code'].unique())

[155]


In [13]:
print(df['houshiki_code'].unique())

[10 90 92]


In [27]:
print(df['teikeikaisha_code1_2'].unique())

['D0001' 'D0002' 'D0004' 'D0005' 'D0006' 'D0008' 'D0011' 'D0014' 'D0018'
 'D0021' 'D0022' 'D0023' 'D0024' 'D0050' 'D0101' 'D0102' 'D0104' 'D0105'
 'D0106' 'D0108' 'D0111' 'D0114' 'D0118' 'D0201' 'D0204' 'D0225' 'D0226'
 'D0227' 'D0228' 'D0003' 'D0095']


In [28]:
print(df['card_code'].unique())

[ 2 31  0]


In [29]:
print(df['brand_code'].unique())

[1 2 0]


In [30]:
print(df['buten_code'].unique())

[0]


In [31]:
print(df['nendo'].unique())

[2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019
 2020 2005 1976 1980 1982 1988 1989 1991 1992 1994 1997 1998 1999 2000
 2001 2002 2003 2004 1971 1973 1975 1981 1983 1984 1985 1986 1987 1990
 1993 1995 1996 1979 1972 1974 1977 1978]


In [32]:
print(df['job_code'].unique())

['3' ' ' '0' '1' '4' '5' '9' '2' 3 4 5 9 0 1 2]


In [54]:
df['job_code'] = df['job_code'].astype('str')

In [55]:
print(df['job_code'].unique())

['3' ' ' '0' '1' '4' '5' '9' '2']


In [33]:
print(df['age_code'].unique())

[ 7  4  5  6  8  9 10 11 12 13 14 15  3  2]


In [177]:
age_list = pd.read_csv(data_dir/f"{input_file2}.csv",encoding='shift-jis')
age_list

,age_code,age_range
0,1,0歳-17歳
1,2,18歳-19歳
2,3,20歳-24歳
3,4,25歳-29歳
4,5,30歳-34歳
5,6,35歳-39歳
6,7,40歳-44歳
7,8,45歳-49歳
8,9,50歳-54歳
9,10,55歳-59歳


In [178]:
df = pd.merge(df,age_list, on =['age_code'], how = 'left')

In [16]:
print(df['age_range'].unique())

['40歳-44歳' '25歳-29歳' '30歳-34歳' '35歳-39歳' '45歳-49歳' '50歳-54歳' '55歳-59歳'
 '60歳-64歳' '65歳-69歳' '70歳-74歳' '75歳-79歳' '80歳以上' '20歳-24歳' '18歳-19歳']


In [34]:
print(df['gender_code'].unique())

[2 1 0]


In [35]:
print(df['cp'].unique())

[201903 201904 201905 201906 201907 201908 201909 201910 201911 201912
 202001 202002 202003 202004 202005 202006 202007]


In [179]:
_df = df.copy()

In [180]:
#日付データの変換
_df['cp_date'] = pd.to_datetime(_df['cp'], format="%Y%m") + MonthEnd(1)
_df['bop_date'] = pd.to_datetime(_df['cp'], format="%Y%m") + MonthEnd(2)

In [181]:
_df['cp_date'] = _df['cp_date'].dt.strftime('%Y/%m/%d')
_df['bop_date'] = _df['bop_date'].dt.strftime('%Y/%m/%d')

In [182]:
print(_df['cp_date'].unique())

['2019/03/31' '2019/04/30' '2019/05/31' '2019/06/30' '2019/07/31'
 '2019/08/31' '2019/09/30' '2019/10/31' '2019/11/30' '2019/12/31'
 '2020/01/31' '2020/02/29' '2020/03/31' '2020/04/30' '2020/05/31'
 '2020/06/30' '2020/07/31']


In [183]:
print(_df['bop_date'].unique())

['2019/04/30' '2019/05/31' '2019/06/30' '2019/07/31' '2019/08/31'
 '2019/09/30' '2019/10/31' '2019/11/30' '2019/12/31' '2020/01/31'
 '2020/02/29' '2020/03/31' '2020/04/30' '2020/05/31' '2020/06/30'
 '2020/07/31' '2020/08/31']


In [184]:
df = _df.copy()

In [185]:
#当月末
q = """
        SELECT
        cp_date
        ,case
            when substr(teikeikaisha_code1_2,1,3) = 'D02' then '外商'
            when substr(teikeikaisha_code1_2,4,5) = '05' then 'ハウスゴールド'
            when card_code = 2 then 'ゴールド'
            else '一般'
        end as card_cate
        ,age_range
        ,case 
            when gender_code = 1 then '男性'
            when gender_code = 2 then '女性'
            else '不明'
        end as gender        
        ,sum(active_account) as eop_no_account
        ,sum(cancel_account) as sum_no_cancel
        from 
            df
        where 
            substr(teikeikaisha_code1_2,4,5) not in ('95','50')
        group by
            1,2,3,4
        """
df_eop = sqldf(q, locals())
df_eop.head()

,cp_date,card_cate,age_range,gender,eop_no_account,sum_no_cancel
0,2019/03/31,ゴールド,18歳-19歳,女性,0,0
1,2019/03/31,ゴールド,18歳-19歳,男性,0,0
2,2019/03/31,ゴールド,20歳-24歳,女性,0,0
3,2019/03/31,ゴールド,20歳-24歳,男性,0,0
4,2019/03/31,ゴールド,25歳-29歳,女性,715,10


In [186]:
#前月末
q = """
        SELECT
        bop_date as cp_date
        ,case
            when substr(teikeikaisha_code1_2,1,3) = 'D02' then '外商'
            when substr(teikeikaisha_code1_2,4,5) = '05' then 'ハウスゴールド'
            when card_code = 2 then 'ゴールド'
            else '一般'
        end as card_cate
        ,age_range
        ,case 
            when gender_code = 1 then '男性'
            when gender_code = 2 then '女性'
            else '不明'
        end as gender        
        ,sum(active_account) as bop_no_account
        from 
            df
        where 
            substr(teikeikaisha_code1_2,4,5) not in ('95','50')
        group by
            1,2,3,4
        """
df_bop = sqldf(q, locals())

In [187]:
#マージ
q = """
        SELECT
        t1.cp_date
        ,t1.card_cate
        ,t1.age_range
        ,t1.gender
        ,t1.eop_no_account
        ,t1.sum_no_cancel
        ,t2.bop_no_account
        from 
            df_eop t1
        left join 
            df_bop t2
        on t1.cp_date = t2.cp_date
        and t1.card_cate = t2.card_cate
        and t1.age_range = t2.age_range
        and t1.gender = t2.gender
        order by
            1,2,3,4
        """
df_eop2 = sqldf(q, locals())
df_eop2

,cp_date,card_cate,age_range,gender,eop_no_account,sum_no_cancel,bop_no_account
0,2019/03/31,ゴールド,18歳-19歳,女性,0,0,NaN
1,2019/03/31,ゴールド,18歳-19歳,男性,0,0,NaN
2,2019/03/31,ゴールド,20歳-24歳,女性,0,0,NaN
3,2019/03/31,ゴールド,20歳-24歳,男性,0,0,NaN
4,2019/03/31,ゴールド,25歳-29歳,女性,715,10,NaN
...,...,...,...,...,...,...,...
2188,2020/07/31,外商,70歳-74歳,男性,14888,58,14817.0
2189,2020/07/31,外商,75歳-79歳,女性,4700,20,4679.0
2190,2020/07/31,外商,75歳-79歳,男性,10125,41,10097.0
2191,2020/07/31,外商,80歳以上,女性,9767,81,9765.0


In [188]:
df_eop2['cancel_rate'] = df_eop2['sum_no_cancel']/df_eop2['bop_no_account']
df_eop2

,cp_date,card_cate,age_range,gender,eop_no_account,sum_no_cancel,bop_no_account,cancel_rate
0,2019/03/31,ゴールド,18歳-19歳,女性,0,0,NaN,NaN
1,2019/03/31,ゴールド,18歳-19歳,男性,0,0,NaN,NaN
2,2019/03/31,ゴールド,20歳-24歳,女性,0,0,NaN,NaN
3,2019/03/31,ゴールド,20歳-24歳,男性,0,0,NaN,NaN
4,2019/03/31,ゴールド,25歳-29歳,女性,715,10,NaN,NaN
...,...,...,...,...,...,...,...,...
2188,2020/07/31,外商,70歳-74歳,男性,14888,58,14817.0,0.003914
2189,2020/07/31,外商,75歳-79歳,女性,4700,20,4679.0,0.004274
2190,2020/07/31,外商,75歳-79歳,男性,10125,41,10097.0,0.004061
2191,2020/07/31,外商,80歳以上,女性,9767,81,9765.0,0.008295


In [189]:
#CSV出力
df_eop2.to_csv(data_dir /'集計_解約数速報時系列_from_cognos.csv', header=True ,index=None, encoding='shift-jis')